In [11]:
import os
import librosa
import numpy as np
from PIL import Image
import librosa.display
from librosa import feature
import matplotlib.pyplot as plt


In [12]:
SAMPLE_RATE = 16000
FRAME_SIZE = 2048
HOP_SIZE = FRAME_SIZE // 4
SEGMENT_LENGTH = SAMPLE_RATE * 2
SEGMENT_HOP = SEGMENT_LENGTH // 2

In [13]:
input_path = "../data/raw-audio/gunshots"
output_path = "../data/unpartitioned-images/gunshots"

print(os.path.exists(input_path))
print(os.path.exists(output_path))

True
True


In [14]:
def mel_spectrogram_generator(path, save_path = "", sr = 16000, duration = 2.0, n_fft = 2560, hop_length = 128, n_mels = 512, fmin = 4000, fmax = 8000, power = 2.0, figsize = (5,5), target_shape = (256, 256), show = False, save = False):
    # Load audio data from path
    data, sr = librosa.load(path, sr = sr, duration = duration)

    # Compute spectrogram
    spectrogram = librosa.feature.melspectrogram(
        y = data,
        sr = sr,
        n_fft = n_fft,
        hop_length = hop_length,
        n_mels = n_mels,
        fmin = fmin,
        fmax = fmax,
        power = power
    )

    # Convert to decibel
    spectrogram_decibel = librosa.power_to_db(spectrogram)

    # Open and configure plot
    fig, ax = plt.subplots(figsize=figsize, dpi=100)
    ax.set_position([0, 0, 1, 1])
    ax.set_frame_on(False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    fig.patch.set_alpha(0)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.ioff()

    # Plot spectrogram
    librosa.display.specshow(
        spectrogram_decibel,
        sr = sr,
        hop_length = hop_length,
        x_axis = "time",
        y_axis = "mel",
        fmin = fmin,
        fmax = fmax,
        vmin = -20,
        vmax = 10,
        cmap = 'magma'
    )

    # Copy spectrogram graph to image, then delete graph
    fig.canvas.draw()
    image = Image.frombytes('RGB', fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
    plt.close(fig)

    # Resize image to CNN input layer
    image = image.resize(target_shape)

    # When True: Image of spectrogram is saved to cwd
    if save:
        image.save(save_path)

    # When False: Return numpy array for CNN input
    if not show:

        # Convert to array
        image_array = np.array(image)

        # Normalize [0, 1]
        image_array = image_array.astype(np.float32) / 255.0

        return image_array

    # When True: Print spectrogram to console
    else:
        image.show()

In [15]:
# Oh good lord...
print(len(os.listdir(output_path)))

0


In [16]:
print(f"{output_path}/gunshot_{0}.png")

../data/unpartitioned-images/gunshots/gunshot_0.png


In [17]:
stop_at = 1
count = 0

# Loop through files in input_path directory
for file in sorted(os.listdir(input_path)):
    # Only process x amount of files
    if count == stop_at:
        break

    # Open file path
    file_path = os.path.join(input_path, file)
    file_name = f"{output_path}/{file[:-4]}.png"

    # Generate spectrogram
    mel_spectrogram_generator(file_path, save_path = file_name, save = True)

    # Increment processed count
    count += 1

EOFError: 